<a href="https://colab.research.google.com/github/JuanCervigon/Globalizacion/blob/main/Agrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

A partir de la tabla de Figaro en formato largo csv, se agregan ramas de actividad y se guarda el resultado en un excel en formato plano y otro en formato matricial

In [ ]:
# Importar librerías necesarias
import pandas as pd

# Conectar con Drive, leer figaro y leer mapeo


In [ ]:
# Conectar con Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Leer figaro en un dataframe [9363300 rows × 7 columns]
# Descargar previamente en Drive desde Eurostat, formato csv versión larga o flat

ruta='/content/drive/MyDrive/Figaro/flatfile_eu-ic-io_ind-by-ind_23ed_2020.csv'
figaro=pd.read_csv(ruta)


In [ ]:
# Mapear las variables refArea,	rowIi, counterpartArea,	colIi


# Leer las agrupaciones
fgeo=pd.read_excel('/content/drive/MyDrive/Figaro/m_es_resto.xlsx','fgeo').set_index('cod')
fnace=pd.read_excel('/content/drive/MyDrive/Figaro/m_es_resto.xlsx','fnace').set_index('cod')
cgeo=pd.read_excel('/content/drive/MyDrive/Figaro/m_es_resto.xlsx','cgeo').set_index('cod')
cnace=pd.read_excel('/content/drive/MyDrive/Figaro/m_es_resto.xlsx','cnace').set_index('cod')

# Convertir las agrupaciones en diccionarios

fgeo=dict(fgeo.to_dict()['map'])
fnace=dict(fnace.to_dict()['map'])
cgeo=dict(cgeo.to_dict()['map'])
cnace=dict(cnace.to_dict()['map'])

# Crear las filas y columnas de la matriz

figaro['filas']=figaro.refArea.map(fgeo)+'_'+figaro.rowIi.map(fnace)
figaro['columnas']=figaro.counterpartArea.map(cgeo)+'_'+figaro.colIi.map(cnace)


# Crear la matriz y pasar a excel

In [ ]:
# Agregar los valores por las filas y columnas de la matriz

figaro_matriz=figaro.pivot_table(values='obsValue', index="filas", columns="columnas", aggfunc='sum',sort=False)

# Guardar la matriz en un fichero excel

figaro_matriz.to_excel('/content/drive/MyDrive/Figaro/f_es_resto.xlsx')


In [ ]:
figaro_matriz

columnas,Rw_CI,ES_CI,Rw_P3_S13,Rw_P3_S14,Rw_P3_S15,Rw_P51G,Rw_P5M,ES_P3_S13,ES_P3_S14,ES_P3_S15,ES_P51G,ES_P5M
filas,,,,,,,,,,,,
Rw_CI,7.297178e+07,209841.544,1.307465e+07,3.782357e+07,1123933.870,1.872033e+07,430966.335,7144.186,65859.785,0.0,24736.210,3170.382
ES_CI,1.960305e+05,768180.696,3.045863e+03,7.667046e+04,58.568,3.746649e+04,5891.773,238506.564,489076.172,12854.0,190732.961,-2678.400
W2_D21X31,1.406226e+06,17729.335,3.961837e+04,1.975827e+06,484.620,4.084942e+05,-85717.147,695.213,67403.702,0.0,12095.813,0.000
W2_OP_RES,0.000000e+00,0.000,0.000000e+00,4.088321e+05,0.000,0.000000e+00,0.000,0.000,6161.172,0.0,0.000,0.000
W2_OP_NRES,0.000000e+00,0.000,0.000000e+00,-4.009384e+05,0.000,0.000000e+00,0.000,0.000,-14054.832,0.0,0.000,0.000
W2_D1,3.637487e+07,555711.332,0.000000e+00,0.000000e+00,0.000,0.000000e+00,0.000,0.000,0.000,0.0,0.000,0.000
W2_D29X39,2.015373e+06,3998.125,0.000000e+00,0.000000e+00,0.000,0.000000e+00,0.000,0.000,0.000,0.0,0.000,0.000
W2_B2A3G,3.149172e+07,460374.805,0.000000e+00,0.000000e+00,0.000,0.000000e+00,0.000,0.000,0.000,0.0,0.000,0.000


# Extraer matrices

In [ ]:
import numpy as np
from numpy.linalg import inv

In [ ]:
# Calculo de la matriz de leontief B
a=df.to_numpy() # convertir el data frame en un array
a=a/sum(a) # calcular los coeficientes
A=a[:21,:21] # extraer los coeficientes de loc Consumos Intermedios
I=np.identity(len(A[0])) # crear la matriz identidad
B=np.linalg.inv(I-A) # calcular la matriz de leontief B=inversa(I-A)
B
